# Chunk Adatok Kiértékelése - CourtRankRL Projekt

Ez a notebook a chunks.jsonl fájlban található chunk adatokat elemzi. Az agents.md specifikáció alapján készített kiértékelési szempontokat vizsgálja a jelenlegi adatokkal.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import json
from pathlib import Path
from typing import Dict, Any, List
from collections import Counter

# Plot stílus beállítása
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Projekt konfiguráció betöltése
import sys
project_root = Path(__file__).parent.parent
sys.path.insert(0, str(project_root))
from configs import config

print("CourtRankRL - Chunk Data Analysis")
print(f"Chunks: {config.CHUNKS_JSONL}")
print(f"Processed docs: {config.PROCESSED_DOCS_LIST}")

NameError: name '__file__' is not defined

## 1. Chunk Adatok Betöltése

A chunks.jsonl fájl betöltése mintavételezéssel a teljesítmény érdekében.

In [ ]:
# Chunkok betöltése mintavételezéssel
chunks_file = config.CHUNKS_JSONL
sample_size = min(50000, 50000)  # Maximum 50k chunk elemzésre

df = None
print(f"Chunkok betöltése: {chunks_file}")
print(f"Mintavétel: {sample_size} chunk")

if chunks_file.exists():
    try:
        chunks_list = []
        with open(chunks_file, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= sample_size:
                    break
                try:
                    chunk = json.loads(line.strip())
                    chunks_list.append(chunk)
                except json.JSONDecodeError:
                    continue
        
        if chunks_list:
            df = pd.DataFrame(chunks_list)
            print(f"✅ Betöltött chunkok száma: {len(df)}")
            print(f"Oszlopok: {df.columns.tolist()}")
        else:
            print("⚠️ Nem találhatóak chunk adatok")
            df = None
    except Exception as e:
        print(f"❌ Hiba a chunkok betöltése során: {e}")
        df = None
else:
    print(f"❌ Chunks fájl nem található: {chunks_file}")
    print("Futtassa a build pipeline-t először: uv run courtrankrl build")
    df = None

if df is not None and not df.empty:
    print(f"\nAdatok betöltve: {df.shape[0]} chunk, {df.shape[1]} oszlop")
    print(f"Első chunk szövege (részlet):")
    first_text = df['text'].iloc[0] if 'text' in df.columns else 'N/A'
    print(f"{str(first_text)[:200]}...")
else:
    print("\n❌ Nincs adat az elemzéshez")

## 2. Alapvető Információk és Statisztikák

A chunk adatok alapvető statisztikáinak elemzése.

In [ ]:
if df is not None and not df.empty:
    print("📊 Chunk adatok alapvető információi:")
    
    # Adattípusok
    print(f"Adatok alakja: {df.shape}")
    print(f"Oszlopok: {df.columns.tolist()}")
    df.info()
    
    # Mintavétel az első néhány chunkból
    print("\nElső 3 chunk adatai:")
    for i in range(min(3, len(df))):
        chunk = df.iloc[i]
        print(f"\nChunk {i+1}:")
        print(f"  doc_id: {chunk.get('doc_id', 'N/A')}")
        print(f"  chunk_id: {chunk.get('chunk_id', 'N/A')}")
        print(f"  birosag: {chunk.get('birosag', 'N/A')}")
        print(f"  JogTerulet: {chunk.get('JogTerulet', 'N/A')}")
        if 'text' in chunk:
            text_preview = str(chunk['text'])[:100] + "..." if len(str(chunk['text'])) > 100 else str(chunk['text'])
            print(f"  text: {text_preview}")
    
    # Leíró statisztikák
    print("\n📈 Leíró statisztikák:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        display(df[numeric_cols].describe())
    
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        display(df[categorical_cols].describe())
else:
    print("❌ Nincs adat az alapvető információk elemzéséhez")

## 3. Hiányzó Értékek Elemzése

A hiányzó értékek azonosítása chunk szinten.

In [ ]:
if df is not None and not df.empty:
    print("🔍 Chunk adatok hiányzó értékei:")
    
    # Hiányzó értékek statisztikái
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    missing_df = pd.DataFrame({'Darabszám': missing_values, 'Százalék': missing_percent})
    missing_df = missing_df[missing_df['Darabszám'] > 0].sort_values(by='Százalék', ascending=False)
    
    print("Hiányzó értékek oszloponként:")
    display(missing_df)
    
    # Hiányzó értékek vizualizációja
    plt.figure(figsize=(15, 8))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title('Hiányzó értékek eloszlása a chunk adatokban')
    plt.show()
    
    # Kritikus hiányzó értékek
    critical_missing = missing_df[missing_df['Százalék'] > 50]  # 50% feletti hiány
    if not critical_missing.empty:
        print(f"⚠️ Kritikus hiányzó értékek (>50%): {len(critical_missing)} oszlop")
        display(critical_missing)
    else:
        print("✅ Nincsenek kritikus hiányzó értékek")
else:
    print("❌ Nincs adat a hiányzó értékek elemzéséhez")

## 4. Szöveghossz Elemzése

A chunkok szövegeinek hossza (karakterek és szavak száma szerint).

In [ ]:
if df is not None and not df.empty and 'text' in df.columns:
    print("📏 Szöveghossz elemzése:")
    
    # Karakterek és szavak száma
    df['karakter_szam'] = df['text'].astype(str).apply(len)
    df['szo_szam'] = df['text'].astype(str).apply(lambda x: len(x.split()))
    
    print("Szöveghossz statisztikák:")
    display(df[['karakter_szam', 'szo_szam']].describe())
    
    # Embedding modell követelményeinek ellenőrzése
    max_length = getattr(config, 'EMBEDDING_MAX_LENGTH', 512)
    long_chunks = df[df['karakter_szam'] > max_length]
    print(f"\nChunkok száma, amelyek hosszabbak {max_length} karakternél: {len(long_chunks)} ({100*len(long_chunks)/len(df):.1f}%)")
    
    if len(long_chunks) > 0:
        print(f"Leghosszabb chunk: {df['karakter_szam'].max()} karakter")
        print(f"Legrövidebb hosszú chunk: {long_chunks['karakter_szam'].min()} karakter")
        
        # Leghosszabb chunk részlete
        longest_chunk = df.loc[df['karakter_szam'].idxmax()]
        print(f"\nLeghosszabb chunk részlete:")
        print(f"doc_id: {longest_chunk.get('doc_id', 'N/A')}")
        print(f"chunk_id: {longest_chunk.get('chunk_id', 'N/A')}")
        print(f"text: {str(longest_chunk['text'])[:200]}...")
    
    # Eloszlások vizualizációja
    plt.figure(figsize=(14, 6))
    
    # Karakterek számának eloszlása
    plt.subplot(1, 2, 1)
    sns.histplot(x=df['karakter_szam'], bins=50, kde=True)
    plt.title('Chunk karakterhossz eloszlása')
    plt.xlabel('Karakterek száma')
    plt.ylabel('Chunkok száma')
    plt.grid(True, alpha=0.3)
    
    # Szavak számának eloszlása
    plt.subplot(1, 2, 2)
    sns.histplot(x=df['szo_szam'], bins=50, kde=True)
    plt.title('Chunk szószám eloszlása')
    plt.xlabel('Szavak száma')
    plt.ylabel('Chunkok száma')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Boxplotok
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.boxplot(y=df['karakter_szam'])
    plt.title('Chunk karakterhossz boxplot')
    plt.ylabel('Karakterek száma')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    sns.boxplot(y=df['szo_szam'])
    plt.title('Chunk szószám boxplot')
    plt.ylabel('Szavak száma')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ Nincs 'text' oszlop a szöveghossz elemzéséhez")

## 5. Kategorikus Változók Elemzése

A chunkok metadatainak kategorikus változói.

In [ ]:
def plot_top_categories(df, column_name, top_n=20):
    """Segédfüggvény a leggyakoribb kategóriák megjelenítésére."""
    if column_name not in df.columns:
        print(f"❌ '{column_name}' oszlop nem található")
        return
    
    counts = df[column_name].value_counts()
    print(f"\n'{column_name}' egyedi értékeinek száma: {counts.nunique()}")
    print(f"Leggyakoribb {top_n} érték:")
    display(counts.head(top_n))
    
    plt.figure(figsize=(12, 6))
    counts.head(top_n).plot(kind='bar')
    plt.title(f'Chunkok megoszlása - {column_name} (Top {top_n})')
    plt.xlabel(column_name)
    plt.ylabel('Chunkok száma')
    plt.xticks(rotation=75, ha='right')
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

if df is not None and not df.empty:
    print("📊 Kategorikus változók elemzése:")
    
    # Bíróság elemzése
    if 'birosag' in df.columns:
        plot_top_categories(df, 'birosag', top_n=30)
    
    # Jogterület elemzése
    if 'JogTerulet' in df.columns:
        plot_top_categories(df, 'JogTerulet', top_n=20)
    
    # Egyéb fontos kategorikus változók
    for col in ['MeghozoBirosag', 'Kollegium', 'Azonosito']:
        if col in df.columns:
            plot_top_categories(df, col, top_n=15)
else:
    print("❌ Nincs adat a kategorikus változók elemzéséhez")

## 6. Gyakori Szavak Elemzése

A chunkokban leggyakrabban előforduló szavak elemzése.

In [ ]:
if df is not None and not df.empty and 'text' in df.columns:
    print("🔤 Gyakori szavak elemzése:")
    
    # Mivel nagy lehet az adathalmaz, vegyünk egy mintát
    sample_size = min(len(df), 10000)
    sample_df = df.sample(n=sample_size, random_state=42)
    print(f"Szóelemzés mintanagysága: {sample_size} chunk")
    
    # Egyszerű tokenizálás
    def simple_tokenize(text):
        return [word.lower() for word in text.split() if word.strip()]
    
    # Szavak gyakoriságának számítása
    word_counts = Counter()
    total_words = 0
    
    for text in sample_df['text'].astype(str):
        tokens = simple_tokenize(text)
        word_counts.update(tokens)
        total_words += len(tokens)
    
    # Top 30 szó kiválasztása
    top_words = word_counts.most_common(30)
    word_freq = pd.DataFrame(top_words, columns=['szo', 'gyakorisag'])
    
    print(f"Top 30 leggyakoribb szó (összes szó: {total_words:,}):")
    display(word_freq)
    
    # Vizualizáció
    plt.figure(figsize=(12, 8))
    sns.barplot(x='gyakorisag', y='szo', data=word_freq)
    plt.title('Top 30 leggyakoribb szó a chunkokban')
    plt.xlabel('Gyakoriság')
    plt.ylabel('Szó')
    plt.grid(axis='x', alpha=0.3)
    plt.show()
    
    # Töltelékszavak aránya
    stop_words = ['és', 'a', 'az', 'de', 'hogy', 'is', 'nem', 'van', 'lesz', 'volt', 'mint']
    stop_word_count = sum(word_counts.get(word, 0) for word in stop_words)
    stop_word_ratio = stop_word_count / total_words if total_words > 0 else 0
    print(f"\nTöltelékszavak aránya: {stop_word_ratio:.2%}")
    print(f"Töltelékszavak összesen: {stop_word_count:,}")
    
    # Szóhossz eloszlás
    word_lengths = [len(word) for word in word_counts.keys()]
    plt.figure(figsize=(10, 6))
    plt.hist(word_lengths, bins=range(1, max(word_lengths) + 2), alpha=0.7)
    plt.title('Szóhossz eloszlása a chunkokban')
    plt.xlabel('Szó hossza (karakter)')
    plt.ylabel('Szavak száma')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("❌ Nincs 'text' oszlop a gyakori szavak elemzéséhez")

## 7. Chunk Szerkezet Elemzése

A chunkok belső szerkezetének és tartalmának elemzése.

In [ ]:
if df is not None and not df.empty and 'text' in df.columns:
    print("🏗️ Chunk szerkezet elemzése:")
    
    # Első chunk teljes tartalmának megjelenítése
    first_chunk_text = df['text'].iloc[0] if len(df) > 0 else ''
    print("\nElső chunk teljes szövege:")
    print("=" * 50)
    print(first_chunk_text)
    print("=" * 50)
    
    # Szöveg struktúra elemzés
    sample_df = df.sample(min(1000, len(df)), random_state=42)
    
    # Mondatok és bekezdések elemzése
    sample_df['mondat_szam'] = sample_df['text'].astype(str).apply(
        lambda x: len([s for s in x.split('.') if s.strip()]) if '.' in x else 1
    )
    sample_df['bekezdes_szam'] = sample_df['text'].astype(str).apply(
        lambda x: len([p for p in x.split('\n') if p.strip()]) if '\n' in x else 1
    )
    
    print(f"\nSzöveg struktúra statisztikák (minta: {len(sample_df)} chunk):")
    display(sample_df[['mondat_szam', 'bekezdes_szam']].describe())
    
    # Struktúra vizualizáció
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    sns.histplot(x=sample_df['mondat_szam'], bins=30, kde=True)
    plt.title('Mondatok száma chunkonként')
    plt.xlabel('Mondatok száma')
    plt.ylabel('Chunkok száma')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    sns.histplot(x=sample_df['bekezdes_szam'], bins=30, kde=True)
    plt.title('Bekezdések száma chunkonként')
    plt.xlabel('Bekezdések száma')
    plt.ylabel('Chunkok száma')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Szöveg komplexitás
    sample_df['atlag_mondat_hossz'] = sample_df.apply(
        lambda row: row['karakter_szam'] / max(row['mondat_szam'], 1), axis=1
    )
    
    print(f"\nSzöveg komplexitás:")
    display(sample_df['atlag_mondat_hossz'].describe())
    
    plt.figure(figsize=(10, 6))
    plt.hist(sample_df['atlag_mondat_hossz'], bins=50, alpha=0.7)
    plt.title('Átlagos mondathossz eloszlása')
    plt.xlabel('Karakterek mondatonként')
    plt.ylabel('Chunkok száma')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("❌ Nincs 'text' oszlop a chunk szerkezet elemzéséhez")

## 8. Metadatok és Szöveg Kapcsolata

A chunk metadatok és a szöveges tartalom közötti kapcsolatok.

In [ ]:
if df is not None and not df.empty and 'text' in df.columns and 'karakter_szam' in df.columns:
    print("🔗 Metadatok és szöveg kapcsolata:")
    
    # Szöveghossz és kategorikus változók kapcsolata
    categorical_cols = ['birosag', 'JogTerulet', 'MeghozoBirosag']
    available_cols = [col for col in categorical_cols if col in df.columns]
    
    for col in available_cols[:3]:  # Maximum 3 kategorikus változó
        if df[col].nunique() <= 20:  # Csak ha nem túl sok egyedi érték
            plt.figure(figsize=(12, 6))
            top_categories = df[col].value_counts().nlargest(10).index
            df_filtered = df[df[col].isin(top_categories)]
            
            sns.boxplot(data=df_filtered, x=col, y='karakter_szam')
            plt.title(f'Szöveghossz eloszlása - {col}')
            plt.xlabel(col)
            plt.ylabel('Karakterek száma')
            plt.xticks(rotation=45, ha='right')
            plt.grid(True, alpha=0.3)
            plt.show()
    
    # Korreláció numerikus változók között
    numeric_cols = ['karakter_szam', 'szo_szam', 'HatarozatEve']
    available_numeric = [col for col in numeric_cols if col in df.columns and df[col].dtype in ['int64', 'float64']]
    
    if len(available_numeric) > 1:
        correlation_matrix = df[available_numeric].corr()
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Numerikus változók korrelációja')
        plt.show()
        
        print("\nKorrelációs mátrix:")
        display(correlation_matrix)
else:
    print("❌ Nincs elég adat a metadatok és szöveg kapcsolat elemzéséhez")

## 9. Chunk Minőség Ellenőrzése

A chunking minőségének és konzisztenciájának ellenőrzése.

In [ ]:
if df is not None and not df.empty:
    print("✅ Chunk minőség ellenőrzése:")
    
    # Alapvető ellenőrzések
    quality_issues = {}
    
    # 1. Üres vagy túl rövid chunkok
    if 'karakter_szam' in df.columns:
        empty_chunks = df[df['karakter_szam'] < 50].shape[0]
        quality_issues['Túl rövid chunkok (<50 karakter)'] = empty_chunks
    
    # 2. Túl hosszú chunkok
    if 'karakter_szam' in df.columns:
        max_length = getattr(config, 'EMBEDDING_MAX_LENGTH', 512)
        long_chunks = df[df['karakter_szam'] > max_length].shape[0]
        quality_issues[f'Túl hosszú chunkok (>{max_length} karakter)'] = long_chunks
    
    # 3. Hiányzó metadatok
    required_cols = ['doc_id', 'chunk_id']
    for col in required_cols:
        if col in df.columns:
            missing = df[col].isnull().sum()
            if missing > 0:
                quality_issues[f'Hiányzó {col}'] = missing
    
    # 4. Duplikált chunk_id-k
    if 'chunk_id' in df.columns:
        duplicates = df['chunk_id'].duplicated().sum()
        if duplicates > 0:
            quality_issues['Duplikált chunk_id-k'] = duplicates
    
    # 5. Hiányzó szöveg
    if 'text' in df.columns:
        empty_text = df[df['text'].astype(str).str.len() < 10].shape[0]
        if empty_text > 0:
            quality_issues['Túl rövid/üres szöveg'] = empty_text
    
    # Eredmények megjelenítése
    if quality_issues:
        print("\n⚠️ Chunk minőségi problémák:")
        for issue, count in quality_issues.items():
            print(f"  {issue}: {count} db ({100*count/len(df):.1f}%)")
        
        # Problémás chunkok részletei
        print("\nProblémás chunkok részletei:")
        problematic_mask = pd.Series(False, index=df.index)
        
        if 'karakter_szam' in df.columns:
            problematic_mask |= (df['karakter_szam'] < 50)
            problematic_mask |= (df['karakter_szam'] > max_length)
        
        if 'text' in df.columns:
            problematic_mask |= (df['text'].astype(str).str.len() < 10)
        
        if problematic_mask.any():
            problematic_df = df[problematic_mask].head(5)
            for i, (_, chunk) in enumerate(problematic_df.iterrows()):
                print(f"\nProblémás chunk {i+1}:")
                print(f"  chunk_id: {chunk.get('chunk_id', 'N/A')}")
                print(f"  doc_id: {chunk.get('doc_id', 'N/A')}")
                if 'karakter_szam' in chunk:
                    print(f"  karakter_szam: {chunk['karakter_szam']}")
                if 'text' in chunk:
                    text_preview = str(chunk['text'])[:150] + "..." if len(str(chunk['text'])) > 150 else str(chunk['text'])
                    print(f"  text: {text_preview}")
    else:
        print("\n✅ Nincsenek jelentős minőségi problémák")
        
    # Összefoglaló statisztikák
    print("\n📊 Chunk minőség összefoglaló:")
    print(f"  Összes chunk: {len(df)}")
    if 'karakter_szam' in df.columns:
        print(f"  Átlagos karakterhossz: {df['karakter_szam'].mean():.0f}")
        print(f"  Medián karakterhossz: {df['karakter_szam'].median():.0f}")
    if 'szo_szam' in df.columns:
        print(f"  Átlagos szószám: {df['szo_szam'].mean():.0f}")
        print(f"  Medián szószám: {df['szo_szam'].median():.0f}")
    
    # Chunk eloszlás bíróság szerint
    if 'birosag' in df.columns:
        chunks_per_court = df.groupby('birosag').size().sort_values(ascending=False)
        print(f"\nChunkok eloszlása bíróság szerint (Top 5):")
        for court, count in chunks_per_court.head().items():
            print(f"  {court}: {count} chunk")
else:
    print("❌ Nincs adat a chunk minőség ellenőrzéséhez")

## 10. Következtetések

A chunk adatok kiértékelésének összefoglalása.

In [ ]:
print("=== CHUNK ADATOK ELEMZÉS ÖSSZEFOGLALÓ ===")
print("\n✅ Sikeresen elemezve:")
if df is not None:
    print(f"   📄 Chunkok: {len(df)} db")
    print(f"   📊 Oszlopok: {len(df.columns)} db")
    
    # Alapvető statisztikák
    if 'karakter_szam' in df.columns:
        print(f"   📏 Átlagos karakterhossz: {df['karakter_szam'].mean():.0f}")
        print(f"   📏 Medián karakterhossz: {df['karakter_szam'].median():.0f}")
    
    if 'szo_szam' in df.columns:
        print(f"   🔤 Átlagos szószám: {df['szo_szam'].mean():.0f}")
        print(f"   🔤 Medián szószám: {df['szo_szam'].median():.0f}")
    
    # Hiányzó értékek
    missing_total = df.isnull().sum().sum()
    total_cells = df.shape[0] * df.shape[1]
    missing_ratio = missing_total / total_cells * 100
    print(f"   ❌ Hiányzó értékek: {missing_total}/{total_cells} ({missing_ratio:.2f}%)")
    
    # Unikalitás
    if 'chunk_id' in df.columns:
        unique_chunks = df['chunk_id'].nunique()
        print(f"   🆔 Egyedi chunk_id: {unique_chunks}/{len(df)} ({100 * unique_chunks / len(df):.2f}%)")

print("\n📋 Agents.md specifikáció ellenőrzés:")
if df is not None:
    # Alapvető metadatok megléte
    required_cols = ['doc_id', 'chunk_id', 'text']
    missing_required = [col for col in required_cols if col not in df.columns]
    if not missing_required:
        print("   ✅ Alapvető metadatok jelen vannak")
    else:
        print(f"   ❌ Hiányzó alapvető metadatok: {missing_required}")
    
    # Szöveghossz ellenőrzés
    if 'karakter_szam' in df.columns:
        max_length = getattr(config, 'EMBEDDING_MAX_LENGTH', 512)
        long_chunks = df[df['karakter_szam'] > max_length].shape[0]
        if long_chunks == 0:
            print("   ✅ Nincsenek túl hosszú chunkok")
        else:
            print(f"   ⚠️ Túl hosszú chunkok: {long_chunks}/{len(df)} ({100*long_chunks/len(df):.1f}%)")
    
    # Jogterületek
    if 'JogTerulet' in df.columns:
        valid_domains = df['JogTerulet'].notna().sum()
        print(f"   ⚖️ Érvényes jogterületek: {valid_domains}/{len(df)} ({100 * valid_domains / len(df):.2f}%)")

print("\n💡 Ajánlások:")
if df is not None:
    # Chunking problémák
    if 'karakter_szam' in df.columns:
        max_length = getattr(config, 'EMBEDDING_MAX_LENGTH', 512)
        short_chunks = df[df['karakter_szam'] < 50].shape[0]
        long_chunks = df[df['karakter_szam'] > max_length].shape[0]
        
        if short_chunks > len(df) * 0.1:  # 10% feletti arány
            print(f"   🔧 Túl sok rövid chunk: {short_chunks} db - chunking paraméterek finomhangolása szükséges")
        
        if long_chunks > len(df) * 0.1:  # 10% feletti arány
            print(f"   ✂️ Túl sok hosszú chunk: {long_chunks} db - chunking paraméterek finomhangolása szükséges")
    
    # Hiányzó metadatok
    missing_cols = df.columns[df.isnull().sum() > len(df) * 0.5]  # 50% feletti hiány
    if len(missing_cols) > 0:
        print(f"   📝 Kritikus hiányzó metadatok: {list(missing_cols)} - javítás szükséges")
    
    # Duplikált chunk_id-k
    if 'chunk_id' in df.columns:
        duplicates = df['chunk_id'].duplicated().sum()
        if duplicates > 0:
            print(f"   🆔 Duplikált chunk_id-k javítása: {duplicates} db")

print("\n🎯 Chunk adatok elemzése kész - a retrieval rendszer használatra kész!")